In [1]:
from langchain.document_loaders import PyPDFLoader
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [ ]:
loader = PyPDFLoader("../../../dataset/article.pdf")
docs = loader.load()
docs

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
)
docs_split = text_splitter.split_documents(docs)
docs_split

In [9]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS



embeddings = OpenAIEmbeddings()

doc_db = FAISS.from_documents(
    documents=docs_split, 
    embedding=embeddings
)

In [ ]:
query = "Prontuário usado em quais países?"
search_docs = doc_db.similarity_search(query)
for doc in search_docs:
    print(f"{doc.page_content}")


In [13]:
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI()

In [15]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=doc_db.as_retriever(),
)

query = "O que é prontuário eletrônico?"
result = qa.run(query)

result

'Prontuário eletrônico é um sistema informatizado que armazena todas as informações sobre a saúde de um paciente, desde o seu histórico médico até resultados de exames e prescrições de medicamentos. Ele é utilizado por profissionais de saúde para acompanhar o tratamento e a evolução do paciente ao longo do tempo. A ideia é que o prontuário eletrônico seja um registro completo e atualizado das informações de saúde do paciente, acessível a qualquer profissional autorizado em qualquer lugar do mundo.'